# 1. 이미지 벡터 추출 함수 정의

In [3]:
#이미지 특징 벡터 함수  추출
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
#코드 출처 https://becominghuman.ai/extract-a-feature-vector-for-any-image-with-pytorch-9717561d1d4c
# https://daeun-computer-uneasy.tistory.com/85

# Load the pretrained model
model = models.resnet18(pretrained=True)

# Use the model object to select the desired layer
layer = model._modules.get('avgpool')

# Set model to evaluation mode
model.eval()

# Image transforms
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)
    img = img.convert('RGB')
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros([1, 512, 1, 512])
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding

c:\Users\minje\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\minje\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
#cosine similarity 함수 정의
def cos_pytorch(A, B):
  cos= nn.CosineSimilarity(dim=1, eps=1e-6)
  cos_sim = cos(A, B)
  return cos_sim.numpy()[0][0][0]

# 2. 인플루언서 리스트 및 파일명 불러오기

In [9]:
import os
#인플루언서 리스트
os.chdir('C:/Korea university/공모전/해커톤/인플루언서정보_샘플')
influencer = os.listdir() #인스타 계정 리스트 저장
#경로 내 파일명 추출
images = {} #key: 계정이름, value: 해당 계정의 이미지 파일명 리스트
for i in influencer:
    os.chdir(f'C:/Korea university/공모전/해커톤/인플루언서정보_샘플/{i}/images')
    files = os.listdir()
    images[i] = files
    
print(influencer)
print(images[influencer[0]])

['@0nefence', '@166.ootd', '@1ungwoo', '@337janggoon', '@59seok', '@98.c_project', '@amourfor_u', '@antmousbe9', '@ap.s_fi1st', '@arcco_iris_', '@bacajini', '@ba_serin_e', '@bbo9ni', '@bejoon0', '@belleofcloset', '@bloggerbok', '@by_he.nique', '@campo_look', '@chaileeson', '@c_eunnnnnnn', '@dbs.ycaa', '@dismas_', '@dosirak_hansol', '@d_soms', '@eungil_j', '@e_wolly', '@geol_dong', '@grey_woo9', '@hamnihouse', '@hawl_0.s', '@hmm.__.u', '@hodu__jwan', '@hotneul', '@hwi____ii', '@j0ng_wo0', '@jelly_wony', '@jess.02.23', '@jin.wonder', '@jindalorian', '@jin_pyo_is', '@jung_staas', '@kimchiz_man', '@kimyannnnngh', '@kj_m.w', '@kxyxn', '@k_8_8bsoo', '@lamode.seoul', '@lil_0uzi_vert', '@loolinmx', '@lxx.s.y_', '@malko_bee', '@mavlfit', '@mini.d31', '@moodonx2', '@my_own_way_____', '@m_n__day', '@odor_bubu', '@oneh6_', '@one_r_k', '@oohsehun', '@oytoyt_', '@ro.seon', '@rozley._.y', '@rupinydaily', '@ruri.kim', '@s.s_jun', '@shura_twins_korea', '@siaestival', '@skuukzky', '@slglf', '@so_h_appy'

In [10]:
import pandas as pd
influencer_df = pd.DataFrame()
influencer_df['id'] = influencer
influencer_df['img_name'] = [images[influencer[i]] for i in range(len(influencer))]
influencer_df.to_csv('C:/Korea university/공모전/해커톤/모델링/업체-인플루언서/influencer_df.csv')

# 3. 피드 이미지 유사도를 통한 인플루언서 추천함수 정의

Resnet18로 추출한 피드 이미지의 feature vector 간의 코사인 유사도를 측정하여 소비자의 피드이미지와 비슷한 색감, 구도를 가진 인플루언서를 추천해준다.

In [60]:
import os
#인플루언서 리스트
os.chdir('C:/Korea university/공모전/해커톤/인플루언서정보_샘플')
influencer = os.listdir()
#경로 내 파일명 추출
images = {}
for i in influencer:
    os.chdir(f'C:/Korea university/공모전/해커톤/인플루언서정보_샘플/{i}/images')
    files = os.listdir()
    images[i] = files

#influencer 샘플 이미지 feature vector 추출 후 리스트로 저장
influencer_features = []
for influen in influencer:
    influen_features =[]
    for j in images[influen]:
        feature = get_vector(f'C:/Korea university/공모전/해커톤/인플루언서정보_샘플/{influen}/images/{j}')
        influen_features.append(feature)
    influencer_features.append(influen_features)

In [ ]:
#인플루언서 피드와 평균 cos similarity 계산하여 유사도가 가장 높은 인플루언서 아이디와 유사도를 도출하는 함수
#input_images: 계산할 이미지의 경로 리스트
def recommend_for_consumer(input_images):
    #input 이미지 feature vector 추출
    input_features=[]
    for i in input_images:
        feature = get_vector(i)
        input_features.append(feature)

    #평균 cos similarity 계산
    sims_total = []
    for k in range(len(influencer_features)): #여러 인플루언서들의 feature vector
        sims = []
        for a in input_features: #input 이미지의 feature vector
            for b in influencer_features[k]: #하나의 인플루언서의 feature vector
                sim = cos_pytorch(a,b)
                sims.append(sim)
        sim_average = sum(sims)/len(sims)
        sims_total.append((sim_average, k)) #평균 유사도와 인플루언서 index 저장
    
    result = sorted(sims_total, reverse=True)
    result = result[:10]

    influencer_recommend = [(influencer[result[i][1]],result[i][0]) for i in range(10)]

    return influencer_recommend

일반인 피드 샘플로 test 해보기

In [21]:
input = [f'C:/Korea university/공모전/해커톤/소비자 피드 샘플/김수경/{i+1}.PNG' for i in range(6)] #test 
test_result = recommend_for_consumer(input)
test_result

[('@velyjuu', 0.6648282972283852),
 ('@so_love_so_', 0.6633930996060371),
 ('@__v.yuum_look__', 0.6594102927323046),
 ('@bacajini', 0.6592520656330245),
 ('@shura_twins_korea', 0.6568085376495196),
 ('@campo_look', 0.6563171309301223),
 ('@y._.dulcet', 0.653558206392659),
 ('@by_he.nique', 0.6515352449483341),
 ('@166.ootd', 0.6490226417779923),
 ('@z___meme', 0.6470658401293414)]

In [16]:
input = [f'C:/Korea university/공모전/해커톤/소비자 피드 샘플/lucy/{i+1}.PNG' for i in range(6)] #test 
test_result = recommend_for_consumer(input)
test_result

[('@bacajini', 0.6820959477197557),
 ('@so_love_so_', 0.6798264533281326),
 ('@__v.yuum_look__', 0.6779707635956249),
 ('@velyjuu', 0.676441837579776),
 ('@by_he.nique', 0.6709413775139385),
 ('@166.ootd', 0.6682728164725833),
 ('@y._.dulcet', 0.6670211331711875),
 ('@shura_twins_korea', 0.6636096511358096),
 ('@moodonx2', 0.6624900082747142),
 ('@rozley._.y', 0.6624294946591059)]

In [17]:
input = [f'C:/Korea university/공모전/해커톤/소비자 피드 샘플/이동휘/{i+1}.PNG' for i in range(8)] #test 
test_result = recommend_for_consumer(input)
test_result

[('@kj_m.w', 0.7242736088732878),
 ('@jin_pyo_is', 0.7197259763876597),
 ('@mavlfit', 0.7195060345033805),
 ('@bbo9ni', 0.7187253603977817),
 ('@grey_woo9', 0.7177101150155067),
 ('@j0ng_wo0', 0.7174121186669383),
 ('@__02x02', 0.7166253504653771),
 ('@one_r_k', 0.7161420036492676),
 ('@_jongh0', 0.7141496786581618),
 ('@337janggoon', 0.7117368099590142)]

In [18]:
input = [f'C:/Korea university/공모전/해커톤/소비자 피드 샘플/이병주/{i+1}.PNG' for i in range(10)] #test 
test_result = recommend_for_consumer(input)
test_result

[('@mavlfit', 0.7138312131166458),
 ('@bbo9ni', 0.7063841362084661),
 ('@337janggoon', 0.7047376577059428),
 ('@_jongh0', 0.703956373568092),
 ('@kj_m.w', 0.7017917241652807),
 ('@grey_woo9', 0.7017030203342438),
 ('@j0ng_wo0', 0.7012450058545385),
 ('@yangkoon__dl', 0.7009132476647695),
 ('@sualboys', 0.699509176214536),
 ('@zxcvr0626', 0.6991625875234604)]

In [20]:
input = [f'C:/Korea university/공모전/해커톤/소비자 피드 샘플/스트릿/{i+1}.jpg' for i in range(10)] #test 
test_result = recommend_for_consumer(input)
test_result

[('@jindalorian', 0.6976417964696884),
 ('@so_j2', 0.6859297007322311),
 ('@dbs.ycaa', 0.6814382269101984),
 ('@bbo9ni', 0.6807412200740406),
 ('@zxcvr0626', 0.678497955997785),
 ('@tt__yl', 0.677684428862163),
 ('@hmm.__.u', 0.6773605632781983),
 ('@hodu__jwan', 0.6748948268095653),
 ('@so_h_appy', 0.6747684073448181),
 ('@166.ootd', 0.6739721953868866)]

동일 인플루언서의 피드에 대해 이미지 평균 유사도를 구하여 '비슷한 피드 이미지'라고 할 수 있는 구체적인 유사도 수치(threshold)를 구해본다.

In [8]:
import numpy as np
same_sim=[]
for i in range(len(influencer_features)):
    fea = influencer_features[i]
    sim = []
    for a in fea:
        for b in fea:
            s = cos_pytorch(a,b)
            sim.append(s)
    sim_average = np.mean(sim)
    same_sim.append(sim_average)

print(np.mean(same_sim), min(same_sim), max(same_sim))

0.71576834 0.5643341 0.8427911


동일 인플루언서 피드들의 평균 cos similarity 계산 결과 0.7정도 나온다. 유사도가 최소 0.56부터 최대 0.84까지도 나온다. 이를 기준으로 threshold로 정하면 된다.

In [6]:
input = [f'C:/Korea university/공모전/해커톤/인플루언서정보_샘플/{influencer[0]}/images/{i}' for i in images[influencer[0]]]
threshold_result = recommend_for_consumer(input)
print(influencer[0])
threshold_result

@0nefence


[('@0nefence', 0.7197441086504195),
 ('@166.ootd', 0.7036784715784921),
 ('@__v.yuum_look__', 0.7007498254036081),
 ('@campo_look', 0.6990199821776357),
 ('@bacajini', 0.6908138431253887),
 ('@ro.seon', 0.6848268175125122),
 ('@c_eunnnnnnn', 0.6846637177136209),
 ('@e_wolly', 0.6828418977972533),
 ('@xixxeeonx_4', 0.6823274343874719),
 ('@slglf', 0.6802485388186242)]

당연한 결과이긴 하지만 자기 자신의 피드가 가장 유사도 높게 나온다.

# 4. 색감분석(톤 분류)을 통한 인플루언서 추천함수 정의 (KNN)

각 influencer들의 톤 비율을 계산해둔 데이터셋 불러오기

In [24]:
influencer_tones= pd.read_csv('C:/Korea university/공모전/해커톤/모델링/소비자-인플루언서/tone.csv')
influencer_tones.head()
#tone: [mono, warm, cool] 인스타 피드 중 이미지 톤 비율

,id,tone
0,@0nefence,"[0.3333333333333333, 0.6, 0.06666666666666667]"
1,@166.ootd,"[0.3, 0.7, 0.0]"
2,@1ungwoo,"[0.1724137931034483, 0.6551724137931034, 0.172..."
3,@337janggoon,"[0.06666666666666667, 0.8333333333333334, 0.1]"
4,@59seok,"[0.4827586206896552, 0.3793103448275862, 0.137..."


## 4-1. 이미지 톤 구별함수 정의

In [ ]:
!pip install -U scikit-learn

In [35]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import scipy.misc

# pixel 단위 톤 구별 함수
def rgb_classfy(rgb):
    r = rgb[0]; g = rgb[1]; b = rgb[2]
    
    if ((r/g) >= 0.95 and (r/g) <= 1.05) and ((g/b) >= 0.95 and (g/b) <= 1.05) and ((r/g) >= 0.95 and (r/g) <= 1.05):
        return 'mono'
    elif r < 51 and g < 51 and b < 51:
        return 'mono'
    else:
        if r > b:
            return 'red'
        else:
            return 'blue'

# image의 톤 구별 함수 (warm, cool, mono)
def image_tone_classifier(image_dir):
    # image의 각 pixel을 rgb 벡터로 저장
    img_array = np.fromfile(image_dir, np.uint8)
    image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    image = cv2.resize(image, dsize=(0, 0), fx=0.2, fy=0.2, interpolation=cv2.INTER_LINEAR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image_list = [list(lst) for img in image for lst in img]
    rb = [rgb_classfy(rgb) for rgb in image_list]
    totalpixel = len(rb)

    mono = rb.count('mono')
    red = rb.count('red')
    blue = rb.count('blue')

    if (red >= blue) & (red >= mono):
        return "warm"
    elif (blue > red) & (blue >= mono):
        return "cool"
    elif (mono > red) & (mono > blue):
        return "mono"

## 4-2. 소비자 input 피드 이미지들에 대해서 톤 비율 도출하는 함수 정의

In [36]:
def get_tone(images): #images: input 이미지들 경로가 담긴 리스트
    mono_cnt = 0
    warm_cnt = 0
    cool_cnt = 0
    img_cnt = len(images)
    for img in images:
        img_tone = image_tone_classifier(img)
        if img_tone == "mono":
            mono_cnt += 1
        elif img_tone == "warm":
            warm_cnt += 1
        elif img_tone == "cool":
            cool_cnt += 1
    tone = [mono_cnt/img_cnt, warm_cnt/img_cnt, cool_cnt/img_cnt] #각 피드이미지에 대한 mono, warm, cool 이미지의 비율 저장
    return tone

test 이미지로 톤 비율 도출해보기

In [37]:
input = [f'C:/Korea university/공모전/해커톤/소비자 피드 샘플/이동휘/{i+1}.PNG' for i in range(8)]
t = get_tone(input)
t

C:\Users\minje\AppData\Local\Temp\ipykernel_1284\3268304376.py:13: RuntimeWarning: divide by zero encountered in ubyte_scalars
  if ((r/g) >= 0.95 and (r/g) <= 1.05) and ((g/b) >= 0.95 and (g/b) <= 1.05) and ((r/g) >= 0.95 and (r/g) <= 1.05):
C:\Users\minje\AppData\Local\Temp\ipykernel_1284\3268304376.py:13: RuntimeWarning: invalid value encountered in ubyte_scalars
  if ((r/g) >= 0.95 and (r/g) <= 1.05) and ((g/b) >= 0.95 and (g/b) <= 1.05) and ((r/g) >= 0.95 and (r/g) <= 1.05):


[0.125, 0.875, 0.0]

In [38]:
input = [f'C:/Korea university/공모전/해커톤/소비자 피드 샘플/lucy/{i+1}.PNG' for i in range(6)]
t = get_tone(input)
t

C:\Users\minje\AppData\Local\Temp\ipykernel_1284\3268304376.py:13: RuntimeWarning: divide by zero encountered in ubyte_scalars
  if ((r/g) >= 0.95 and (r/g) <= 1.05) and ((g/b) >= 0.95 and (g/b) <= 1.05) and ((r/g) >= 0.95 and (r/g) <= 1.05):
C:\Users\minje\AppData\Local\Temp\ipykernel_1284\3268304376.py:13: RuntimeWarning: invalid value encountered in ubyte_scalars
  if ((r/g) >= 0.95 and (r/g) <= 1.05) and ((g/b) >= 0.95 and (g/b) <= 1.05) and ((r/g) >= 0.95 and (r/g) <= 1.05):


[0.3333333333333333, 0.5, 0.16666666666666666]

잘 나온다.

# 5. 3, 4번 함수 융합하여 추천시스템 모델링 구현

이미지 유사도와 색감을 기반으로 소비자 피드 또는 input 이미지와 비슷한 인플루언서를 추천해준다.

In [57]:
#코사인 유사도 계산 함수
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [87]:
#최종 모델링 구현
import ast
def final_recommend_for_consumer(input_images): #input_images: 소비자 input 이미지들의 경로리스트
    #이미지 유사도 기반으로 10순위권 인플루언서 도출
    influencer_top10 = recommend_for_consumer(input_images) #(계정이름, 이미지 유사도) 튜플형식으로 리스트에 저장돼서 나옴

    #이미지 유사도 10위권 인플루언서들의 계정이름과 톤비율 가져오기
    index = []
    imagesim = []
    for i in range(len(influencer_top10)):
        filter = influencer_tones[influencer_tones['id'] == influencer_top10[i][0]].index[0]
        index.append(filter)
        imagesim.append(influencer_top10[i][1])

    influencer_top10_final = influencer_tones.iloc[index, :]
    influencer_top10_final['image_sim'] = imagesim
    influencer_top10_final.reset_index(drop=True, inplace=True)#index 초기화

    #소비자 이미지의 톤 비율 계산
    consumer_tone = get_tone(input_images)

    #소비자 이미지 톤비율의 코사인 유사도가 높은 3개의 인플루언서 도출
    tonesim=[]
    for i in range(len(influencer_top10)):
        sim = cos(consumer_tone, ast.literal_eval(influencer_top10_final['tone'][i])) #ast.literal_eval: str형식으로된 list를 list로 변환
        tonesim.append(sim)
    
    influencer_top10_final['tone_sim'] = tonesim
    influencer_top10_final['total_sim'] = list(influencer_top10_final['image_sim'] + influencer_top10_final['tone_sim']) #이미지 유사도와 색감 각각 같은 가중치로 도출함
    influencer_top10_final.sort_values('total_sim', ascending=False, inplace=True)
    

    return influencer_top10_final.iloc[:3,:]

test이미지 돌려보기

In [82]:
input = [f'C:/Korea university/공모전/해커톤/소비자 피드 샘플/이병주/{i+1}.PNG' for i in range(10)]
test = final_recommend_for_consumer(input)
test

C:\Users\minje\AppData\Local\Temp\ipykernel_1284\417069487.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  influencer_top10_final['image_sim'] = imagesim
C:\Users\minje\AppData\Local\Temp\ipykernel_1284\3268304376.py:13: RuntimeWarning: divide by zero encountered in ubyte_scalars
  if ((r/g) >= 0.95 and (r/g) <= 1.05) and ((g/b) >= 0.95 and (g/b) <= 1.05) and ((r/g) >= 0.95 and (r/g) <= 1.05):
C:\Users\minje\AppData\Local\Temp\ipykernel_1284\3268304376.py:13: RuntimeWarning: invalid value encountered in ubyte_scalars
  if ((r/g) >= 0.95 and (r/g) <= 1.05) and ((g/b) >= 0.95 and (g/b) <= 1.05) and ((r/g) >= 0.95 and (r/g) <= 1.05):
C:\Users\minje\AppData\Local\Temp\ipykernel_1284\417069487.py:28: SettingWithCopyWarning: 
A value is trying to 

,id,tone,image_sim,tone_sim,total_sim
0,@mavlfit,"[0.6, 0.26666666666666666, 0.13333333333333333]",0.713831,0.555170,1.269002
1,@bbo9ni,"[0.17857142857142858, 0.6785714285714286, 0.14...",0.706384,0.990930,1.697314
2,@337janggoon,"[0.06666666666666667, 0.8333333333333334, 0.1]",0.704738,0.985925,1.690663
3,@_jongh0,"[0.35714285714285715, 0.6071428571428571, 0.03...",0.703956,0.902697,1.606654
4,@kj_m.w,"[0.16666666666666666, 0.8, 0.03333333333333333]",0.701792,0.970606,1.672398
5,@grey_woo9,"[0.36666666666666664, 0.4666666666666667, 0.16...",0.701703,0.875663,1.577366
6,@j0ng_wo0,"[0.14285714285714285, 0.6785714285714286, 0.17...",0.701245,0.997717,1.698962
7,@yangkoon__dl,"[0.2, 0.7666666666666667, 0.03333333333333333]",0.700913,0.966677,1.667591
8,@sualboys,"[0.6666666666666666, 0.23333333333333334, 0.1]",0.699509,0.476905,1.176414
9,@zxcvr0626,"[0.13333333333333333, 0.7, 0.16666666666666666]",0.699163,0.997942,1.697105


In [83]:
input = [f'C:/Korea university/공모전/해커톤/소비자 피드 샘플/김수경/{i+1}.PNG' for i in range(6)]
test = final_recommend_for_consumer(input)
test

C:\Users\minje\AppData\Local\Temp\ipykernel_1284\417069487.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  influencer_top10_final['image_sim'] = imagesim
C:\Users\minje\AppData\Local\Temp\ipykernel_1284\3268304376.py:13: RuntimeWarning: divide by zero encountered in ubyte_scalars
  if ((r/g) >= 0.95 and (r/g) <= 1.05) and ((g/b) >= 0.95 and (g/b) <= 1.05) and ((r/g) >= 0.95 and (r/g) <= 1.05):
C:\Users\minje\AppData\Local\Temp\ipykernel_1284\3268304376.py:13: RuntimeWarning: invalid value encountered in ubyte_scalars
  if ((r/g) >= 0.95 and (r/g) <= 1.05) and ((g/b) >= 0.95 and (g/b) <= 1.05) and ((r/g) >= 0.95 and (r/g) <= 1.05):
C:\Users\minje\AppData\Local\Temp\ipykernel_1284\417069487.py:28: SettingWithCopyWarning: 
A value is trying to 

,id,tone,image_sim,tone_sim,total_sim
0,@velyjuu,"[0.07692307692307693, 0.9230769230769231, 0.0]",0.664828,0.891338,1.556166
1,@so_love_so_,"[0.13333333333333333, 0.6666666666666666, 0.2]",0.663393,0.967617,1.631010
2,@__v.yuum_look__,"[0.1724137931034483, 0.7931034482758621, 0.034...",0.659410,0.892209,1.551619
3,@bacajini,"[0.42857142857142855, 0.5, 0.07142857142857142]",0.659252,0.723364,1.382616
4,@shura_twins_korea,"[0.037037037037037035, 0.8518518518518519, 0.1...",0.656809,0.943880,1.600688
5,@campo_look,"[0.27586206896551724, 0.6551724137931034, 0.06...",0.656317,0.863667,1.519984
6,@y._.dulcet,"[0.1, 0.8666666666666667, 0.03333333333333333]",0.653558,0.904959,1.558517
7,@by_he.nique,"[0.03333333333333333, 0.9666666666666667, 0.0]",0.651535,0.893896,1.545431
8,@166.ootd,"[0.3, 0.7, 0.0]",0.649023,0.822108,1.471131
9,@z___meme,"[0.4642857142857143, 0.5, 0.03571428571428571]",0.647066,0.677911,1.324976
